# DEAP library for evolutionary computing

In [65]:
from deap import base, creator
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

/home/sebacastillo/.ealab/lib/python3.9/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/sebacastillo/.ealab/lib/python3.9/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


### Creating an algorithm that match the problem type

In [66]:
import random
from deap import tools

IND_SIZE = 10

toolbox = base.Toolbox()
toolbox.register("attribute", random.random)
toolbox.register("individual", tools.initRepeat, creator.Individual,
                 toolbox.attribute, n=IND_SIZE)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

### Creating individuals

In [67]:
ind1 = toolbox.individual()

In [68]:
print(ind1)               # [0.86..., 0.27..., 0.70..., 0.03..., 0.87...]
print(ind1.fitness.valid) # False

[0.5863350156634686, 0.5243908828442447, 0.27161643071253827, 0.8390459221588656, 0.3944733044040254, 0.30833513415735503, 0.6594789112387072, 0.5743380701875002, 0.677083797315257, 0.8290078925039034]
False


The individual is printed as its base class representation (here a list) and the fitness is invalid because it contains no values.

### Evaluation

In [69]:
def evaluate(individual):
    return sum(individual)/len(individual),

toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluate)

In [70]:
evaluate(ind1)

(0.5664105361185865,)

In [71]:
ind1.fitness.values = evaluate(ind1)  

## Operatos

### Mutation

The general rule for mutation operators is that they only mutate, this means that an independent copy must be made prior to mutating the individual if the original individual has to be kept or is a reference to another individual (see the selection operator).

In order to apply a mutation (here a gaussian mutation) on the individual ind1, simply apply the desired function.

In [72]:
mutant = toolbox.clone(ind1)
ind2, = tools.mutGaussian(mutant, mu=0.0, sigma=0.2, indpb=0.2)
#The fitness’ values are deleted because they’re not related to the individual anymore.
del mutant.fitness.values


In [73]:
print(ind2 is mutant)    # True
print(mutant is ind1)    # False

True
False


In [74]:
ind2

[0.5863350156634686,
 0.5243908828442447,
 0.27161643071253827,
 0.8390459221588656,
 0.3944733044040254,
 0.30833513415735503,
 0.6594789112387072,
 0.5743380701875002,
 0.677083797315257,
 0.8290078925039034]

### Crossover

In [75]:
child1, child2 = [toolbox.clone(ind) for ind in (ind1, ind2)]
tools.cxBlend(child1, child2, 0.5)
del child1.fitness.values
del child2.fitness.values

In [76]:
child1

[0.5863350156634686,
 0.5243908828442447,
 0.27161643071253827,
 0.8390459221588656,
 0.3944733044040254,
 0.30833513415735503,
 0.6594789112387072,
 0.5743380701875002,
 0.677083797315257,
 0.8290078925039034]

In [77]:
child2

[0.5863350156634686,
 0.5243908828442447,
 0.27161643071253827,
 0.8390459221588656,
 0.3944733044040254,
 0.30833513415735503,
 0.6594789112387072,
 0.5743380701875002,
 0.677083797315257,
 0.8290078925039034]

### Selection

In [78]:
selected = tools.selBest([child1, child2], 2)
print(child1 in selected)	# True

True


## Full script

In [79]:
def main():
    pop = toolbox.population(n=50)
    CXPB, MUTPB, NGEN = 0.5, 0.2, 40

    # Evaluate the entire population
    fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit

    for g in range(NGEN):
        # Select the next generation individuals
        offspring = toolbox.select(pop, len(pop))
        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))

        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CXPB:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        # The population is entirely replaced by the offspring
        pop[:] = offspring

    return pop



In [80]:
main()

[[-2.508249975646659,
  -6.616796327807947,
  -2.4807464854930843,
  -4.233641415133933,
  -3.9681471917670206,
  -3.6202693464666877,
  -2.137840432993634,
  -3.449868369773329,
  -1.144061507083703,
  -6.147793613691279],
 [-2.508249975646659,
  -6.616796327807947,
  -2.4807464854930843,
  -4.233641415133933,
  -3.9681471917670206,
  -3.6202693464666877,
  -2.137840432993634,
  -5.038402708320314,
  -1.144061507083703,
  -3.861469517868311],
 [-1.7263501977889546,
  -5.72297819628101,
  -2.4807464854930843,
  -4.233641415133933,
  -3.9681471917670206,
  -3.6202693464666877,
  -2.137840432993634,
  -3.449868369773329,
  -1.144061507083703,
  -6.147793613691279],
 [-2.8568568814992172,
  -5.72297819628101,
  -2.4807464854930843,
  -4.233641415133933,
  -3.9681471917670206,
  -3.6202693464666877,
  -2.137840432993634,
  -3.449868369773329,
  -1.144061507083703,
  -6.147793613691279],
 [-1.7263501977889546,
  -5.72297819628101,
  -2.4807464854930843,
  -4.233641415133933,
  -3.9681471917

## Script Parts

In [81]:
pop = toolbox.population(n=50)
CXPB, MUTPB, NGEN = 0.5, 0.2, 40
print(pop)

[[0.5946172577757016, 0.5119892355813862, 0.12457837267923733, 0.47898821705900285, 0.987196565861196, 0.3190017739375628, 0.7828688791096525, 0.7530182091804029, 0.5203893777854147, 0.055503998378235275], [0.021589489327839773, 0.2987644028642964, 0.8835841452432275, 0.33644164887323547, 0.6066945658918733, 0.18795247355409184, 0.7787270843562082, 0.10619830521898688, 0.41423054079217003, 0.7243276555323906], [0.9677004264477989, 0.5930692647532237, 0.22472543238080422, 0.5857500128755048, 0.3601415275601516, 0.5629526581996005, 0.734887921722173, 0.725235374576175, 0.6405656377466271, 0.19281444386288038], [0.09425431284400698, 0.9128205583910481, 0.62861790714954, 0.3546582598170467, 0.5921160018602927, 0.4856650402804139, 0.895892752737711, 0.015087708345346229, 0.7722359126972432, 0.7213722660490852], [0.589578947980289, 0.01596713858617682, 0.05688181772709133, 0.47124403734273457, 0.10338606053673449, 0.7425044688143417, 0.9187622571856415, 0.45295862417976285, 0.337915435866357

**Opciones**: map(...) vs list(map(...))

In [82]:
# Evaluate the entire population
fitnesses = list(map(toolbox.evaluate, pop))
print(fitnesses)


[(0.5128151887347793,), (0.435851031165432,), (0.5587842700124939,), (0.5472720720171734,), (0.41700765865775835,), (0.525075447153952,), (0.5748767096181572,), (0.3834028886284965,), (0.6176627084443848,), (0.6076305030264917,), (0.4713353493480218,), (0.5781404485676999,), (0.6431490756718391,), (0.5915471494923524,), (0.38831272421389845,), (0.5560235753813589,), (0.48151306326390725,), (0.5251556322835719,), (0.4681107117015825,), (0.3985172003915978,), (0.5437766265814211,), (0.5165887240514844,), (0.47849889780531124,), (0.5376118466613817,), (0.5948898082120893,), (0.44897472715678594,), (0.4671802895097626,), (0.4819348360289986,), (0.4663902799547227,), (0.36206318964575523,), (0.40508318682942335,), (0.43608401601070435,), (0.6262494405392751,), (0.30936122242270975,), (0.480905504366374,), (0.4809095194033645,), (0.31853897316434365,), (0.660284283276502,), (0.44114984613524194,), (0.5569062267016588,), (0.427524195859159,), (0.4397516123631737,), (0.4454660238031571,), (0.4

In [83]:
for ind, fit in zip(pop, fitnesses):
    ind.fitness.values = fit

In [84]:
print(len(pop)), print(len(fitnesses))

50
50


(None, None)

In [85]:
# Evaluate the entire population
fitnesses = map(toolbox.evaluate, pop)
print(fitnesses)

In [86]:
for ind, fit in zip(pop, fitnesses):
    ind.fitness.values = fit

Corre igual en las dos formas, cambia el objeto fitness

Select individuals

In [87]:
# Select the next generation individual
offspring = toolbox.select(pop, len(pop))
len(offspring), print(offspring)

[[0.5946172577757016, 0.5119892355813862, 0.12457837267923733, 0.47898821705900285, 0.987196565861196, 0.3190017739375628, 0.7828688791096525, 0.7530182091804029, 0.5203893777854147, 0.055503998378235275], [0.021589489327839773, 0.2987644028642964, 0.8835841452432275, 0.33644164887323547, 0.6066945658918733, 0.18795247355409184, 0.7787270843562082, 0.10619830521898688, 0.41423054079217003, 0.7243276555323906], [0.11880908768547516, 0.21464233559490575, 0.16710158995802882, 0.8072733917835009, 0.09856788870079158, 0.6993375175956529, 0.5432336812982566, 0.2582600098162656, 0.5357506373520371, 0.954539983846823], [0.048820985399092076, 0.8434210455950211, 0.20388998849648032, 0.06440871897892086, 0.10407858462967456, 0.4394922982925813, 0.934216145497855, 0.6715543431613026, 0.6437951156762955, 0.8553778179365171], [0.3956083313318236, 0.5612648481871038, 0.15977543460969335, 0.14460506682854557, 0.4627404698389205, 0.051866245974154124, 0.024577557339141354, 0.28605231196446634, 0.82377

(50, None)

In [62]:
# Clone the selected individuals
offspring = list(map(toolbox.clone, offspring))
print(offspring)

[[0.594775607976455, 0.5737438603402407, 0.7925686952410342, 0.9673521678675671, 0.8902705693346651, 0.25030232214827763, 0.19496669957117085, 0.10306638390749367, 0.7450412992799142, 0.47773316792604903], [0.5529325912996513, 0.16821303117365938, 0.5408651152739647, 0.6840518770817676, 0.4163890558978057, 0.19704096475510258, 0.2750464232578016, 0.3106890517186448, 0.026679962560018256, 0.7434854339548839], [0.7511648403736367, 0.22285609460238565, 0.5057321838138616, 0.6644065831222871, 0.3828226087051003, 0.17747333298757506, 0.3097955666200991, 0.9292005694846205, 0.16828167819784579, 0.3950141320429269], [0.9084650516813573, 0.4196205681559061, 0.43738175747665786, 0.6270550189475109, 0.2500300653461184, 0.9271940444418659, 0.7474484501811489, 0.20836456004597326, 0.4156403778088572, 0.9328353136717948], [0.7093600370140825, 0.1419503528699666, 0.09822416678557544, 0.05103818703646412, 0.7660750730799439, 0.1413306706632983, 0.5526780000659093, 0.8540585157911679, 0.79251796626245

In [63]:
# Apply crossover and mutation on the offspring
for child1, child2 in zip(offspring[::2], offspring[1::2]):
    if random.random() < CXPB:
        toolbox.mate(child1, child2)
        del child1.fitness.values
        del child2.fitness.values

In [64]:
for mutant in offspring:
    if random.random() < MUTPB:
        toolbox.mutate(mutant)
        del mutant.fitness.values